In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csqa-dataset/csqa_second_stage-5000.csv
/kaggle/input/csqa-logicalcombinations/test_all_hf.json
/kaggle/input/csqa-logicalcombinations/dev_all_hf.json
/kaggle/input/csqa-logicalcombinations/test_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/dev_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/train_logical_combinations_output.json
/kaggle/input/csqa-logicalcombinations/train_all_hf.json
/kaggle/input/csqa-dataset-splits/csqa_dev_set.csv
/kaggle/input/csqa-dataset-splits/csqa_test_set.csv
/kaggle/input/csqa-dataset-splits/csqa_train_set.csv


In [2]:
from datasets import load_dataset

In [3]:
!pip install -q transformers accelerate bitsandbytes peft trl
!pip install -q -U protobuf==5.29.3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [4]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ['HF_TOKEN'] = user_secrets.get_secret("hugging_face")

from huggingface_hub import login
login(new_session=False)

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
import json

def load_and_filter_csqa_data():
    """
    Loads the CommonsenseQA dataset and returns first 5000 rows.
    """
    print("Loading CommonsenseQA dataset...")
    ds = load_dataset("tau/commonsense_qa")
    train_df = ds['train'].to_pandas()
    valid_df = ds['validation'].to_pandas()
    test_df = ds['test'].to_pandas()
    df = pd.concat([train_df, valid_df, test_df], ignore_index=True)
    print(f"Total dataset size: {df.shape}")
    
    # Take first 5000 rows
    df_subset = df.iloc[:5000].copy()
    df_subset.dropna(inplace=True)
    df_subset = df_subset.drop(index=537).reset_index(drop=True)
    print(f"Using {len(df_subset)} questions.")
    return df_subset

def flatten_row(row):
    # Extract choices
    labels = row['choices']['label']       # ['A','B','C','D','E']
    texts  = row['choices']['text']        # actual text
    
    choice_map = {label: text for label, text in zip(labels, texts)}
    
    correct_label = row['answerKey']
    correct_answer_text = choice_map.get(correct_label, None)
    
    return {
        'question': row['question'],
        'A': choice_map.get('A', None),
        'B': choice_map.get('B', None),
        'C': choice_map.get('C', None),
        'D': choice_map.get('D', None),
        'E': choice_map.get('E', None),
        'correct_label': correct_label,
        'correct_answer_text': correct_answer_text
    }


def split_dataset(df):
    test_df = df.sample(n=500, random_state=72)
    remaining_df = df.drop(test_df.index)
    # Step 3. Now it's safe to reset indices for convenience
    test_df = test_df.reset_index(drop=True)
    remaining_df = remaining_df.reset_index(drop=True)
    
    # Step 4. Split remaining into dev and train
    dev_df = remaining_df.sample(n=1500, random_state=72)
    train_df = remaining_df.drop(dev_df.index)
    
    # Optional: reset indexes
    dev_df = dev_df.reset_index(drop=True)
    train_df = train_df.reset_index(drop=True)

    return train_df.apply(flatten_row, axis=1, result_type='expand'), dev_df.apply(flatten_row, axis=1, result_type='expand'), test_df.apply(flatten_row, axis=1, result_type='expand')


In [6]:
df = load_and_filter_csqa_data()

train, dev, test = split_dataset(df)

Loading CommonsenseQA dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Total dataset size: (12102, 5)
Using 4999 questions.


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Load model in 4-bit for QLoRA
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,              # 4-bit quantization
    device_map="auto",              # Automatically split layers to GPU
    bnb_4bit_quant_type="nf4",      # NormalFloat4 (better for QLoRA)
    bnb_4bit_use_double_quant=True, # More compression
    torch_dtype="auto"
)

# Prepare for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=8,            # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

print("Model ready for QLoRA fine-tuning!")

2025-12-01 18:45:52.858970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764614753.031976      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764614753.081715      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model ready for QLoRA fine-tuning!


In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def calculate_qa_metrics(results_df):
    """
    
    Calculate comprehensive evaluation metrics for QA results
    
    Args:
        results_df: DataFrame with 'predicted_label' and 'correct_label' columns
    
    Returns:
        dict: Dictionary containing all evaluation metrics
    """
    
    # Remove None predictions for accurate calculation
    valid_predictions = results_df.dropna(subset=['predicted_label'])
    
    if len(valid_predictions) == 0:
        print("Warning: No valid predictions found!")
        return {}
    
    y_true = valid_predictions['correct_label'].tolist()
    y_pred = valid_predictions['predicted_label'].tolist()
    
    # Basic metrics
    total_questions = len(results_df)
    answered_questions = len(valid_predictions)
    unanswered_questions = total_questions - answered_questions
    
    # Accuracy (main metric for QA)
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    labels = ['A', 'B', 'C', 'D', 'E']
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, average=None, zero_division=0
    )
    
    # Macro and micro averages
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    
    # Answer rate (percentage of questions answered)
    answer_rate = answered_questions / total_questions
    
    # Compile results
    metrics = {
        'total_questions': total_questions,
        'answered_questions': answered_questions,
        'unanswered_questions': unanswered_questions,
        'answer_rate': answer_rate,
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'micro_recall': micro_recall,
        'micro_f1': micro_f1,
    }
    
    # Per-class metrics
    for i, label in enumerate(labels):
        metrics[f'{label}_precision'] = precision[i]
        metrics[f'{label}_recall'] = recall[i]
        metrics[f'{label}_f1'] = f1[i]
        metrics[f'{label}_support'] = support[i]
    
    return metrics, cm, y_true, y_pred

def print_qa_metrics(metrics):
    """Pretty print the evaluation metrics"""
    print("=" * 50)
    print("QA EVALUATION METRICS")
    print("=" * 50)
    
    print(f"Total Questions: {metrics['total_questions']}")
    print(f"Answered Questions: {metrics['answered_questions']}")
    print(f"Answer Rate: {metrics['answer_rate']:.1%}")
    print(f"Accuracy: {metrics['accuracy']:.1%}")
    
    print("\n" + "-" * 30)
    print("OVERALL METRICS")
    print("-" * 30)
    print(f"Macro Precision: {metrics['macro_precision']:.3f}")
    print(f"Macro Recall: {metrics['macro_recall']:.3f}")
    print(f"Macro F1: {metrics['macro_f1']:.3f}")
    print(f"Micro Precision: {metrics['micro_precision']:.3f}")
    print(f"Micro Recall: {metrics['micro_recall']:.3f}")
    print(f"Micro F1: {metrics['micro_f1']:.3f}")
    
    print("\n" + "-" * 40)
    print("PER-CLASS METRICS")
    print("-" * 40)
    print(f"{'Class':<5} {'Precision':<9} {'Recall':<6} {'F1':<6} {'Support'}")
    print("-" * 40)
    
    for label in ['A', 'B', 'C', 'D', 'E']:
        precision = metrics[f'{label}_precision']
        recall = metrics[f'{label}_recall']
        f1 = metrics[f'{label}_f1']
        support = int(metrics[f'{label}_support'])
        print(f"{label:<5} {precision:<9.3f} {recall:<6.3f} {f1:<6.3f} {support}")

def plot_confusion_matrix(cm, labels=['A', 'B', 'C', 'D', 'E'], save_path=None):
    """Plot confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def analyze_answer_distribution(results_df):
    """Analyze the distribution of correct and predicted answers"""
    print("\n" + "=" * 50)
    print("ANSWER DISTRIBUTION ANALYSIS")
    print("=" * 50)
    
    # True label distribution
    true_dist = results_df['correct_label'].value_counts().sort_index()
    print("\nTrue Label Distribution:")
    for label in ['A', 'B', 'C', 'D','E']:
        count = true_dist.get(label, 0)
        pct = count / len(results_df) * 100
        print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Predicted label distribution (excluding None)
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        pred_dist = valid_preds['predicted_label'].value_counts().sort_index()
        print("\nPredicted Label Distribution:")
        for label in ['A', 'B', 'C', 'D','E']:
            count = pred_dist.get(label, 0)
            pct = count / len(valid_preds) * 100
            print(f"  {label}: {count} ({pct:.1f}%)")
    
    # Check for bias towards certain options
    print("\nLabel Rotation Check:")
    if len(set(true_dist.values)) == 1:
        print("✓ Perfect label rotation detected")
    else:
        print("⚠ Uneven label distribution detected")
        
def comprehensive_qa_evaluation(results_df, save_plots=True):
    """Run complete evaluation pipeline"""
    
    # Calculate metrics
    metrics, cm, y_true, y_pred = calculate_qa_metrics(results_df)
    
    if not metrics:
        return
    
    # Print metrics
    print_qa_metrics(metrics)
    
    # Analyze distributions
    analyze_answer_distribution(results_df)
    
    # Plot confusion matrix
    if save_plots:
        plot_confusion_matrix(cm, save_path='confusion_matrix.png')
    else:
        plot_confusion_matrix(cm)
    
    # Print sklearn classification report for additional insights
    valid_preds = results_df.dropna(subset=['predicted_label'])
    if len(valid_preds) > 0:
        print("\n" + "=" * 50)
        print("DETAILED CLASSIFICATION REPORT")
        print("=" * 50)
        print(classification_report(
            valid_preds['correct_label'], 
            valid_preds['predicted_label'],
            labels=['A', 'B', 'C', 'D','E']
        ))
    
    return metrics

In [9]:
def zero_shot_qa(model, tokenizer, qa_dataframe):
    results = []
    required_cols = ['question', 'A', 'B', 'C', 'D', 'E', 'correct_label']
    if not all(col in qa_dataframe.columns for col in required_cols):
        raise ValueError(f"DataFrame must contain the following columns: {required_cols}")
    
    valid_choices = {'A', 'B', 'C', 'D', 'E'}
    
    for idx, row in qa_dataframe.iterrows():
        predicted_label = None
        try:

            prompt = f"""Answer the following commonsense question by selecting the correct option.
            Instructions: Please respond with ONLY the single, capital letter (A, B, C, D or E) that is the correct answer. Do not include any other text, punctuation, or explanation.

Question: {str(row['question'])}

(A) {str(row['A'])}
(B) {str(row['B'])}
(C) {str(row['C'])}
(D) {str(row['D'])}
(E) {str(row['E'])}
Answer:"""

            
            inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
            inputs = {key: value.to(model.device) for key, value in inputs.items()}
            
            # 2. GENERATE the response using the tokenized input
            # We explicitly pass `input_ids` and `attention_mask`
            output_tokens = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_new_tokens=3,
                do_sample=False
            )

            # 3. DECODE the output tokens back into a string
            # We slice the output to get only the newly generated tokens
            generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)


            if "Answer:" in generated_text:
                generated_text = generated_text.split("Answer:")[-1]
            # print(generated_text)
            # Clean up extra prefixes or artifacts
            generated_text = generated_text.strip()  # remove spaces/newlines
            if generated_text.lower().startswith("assistant"):
                generated_text = generated_text.splitlines()[-1].strip()  # keep the last meaningful line

            # 4. Process the generated text for a strict answer
            # Strip all whitespace and common punctuation, then get the first character
            cleaned_text = generated_text.strip().upper().replace('.', '').replace(',', '').replace(' ', '')
            # print(cleaned_text)
            # Validate that the first character is a valid choice
            if cleaned_text and cleaned_text[0] in valid_choices:
                predicted_label = cleaned_text[0]

        except Exception as e:
            # Catch any errors during processing and log them
            print(f"Error processing row {idx}: {e}")
        
        results.append({
            'question': str(row['question']),
            'predicted_label': predicted_label,
            'correct_label': str(row['correct_label']).strip().upper(),
            'full_qa': f"""Question: {str(row['question'])} \n
            A. {str(row['A'])}
            B. {str(row['B'])}
            C. {str(row['C'])}
            D. {str(row['D'])}
            E. {str(row['E'])}"""
        })
            
    return results

In [10]:
results = zero_shot_qa(model, tokenizer, train)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_baseline_results_train.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for op

({'total_questions': 2999, 'answered_questions': 2999, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.7109036345448483, 'macro_precision': 0.712633344205019, 'macro_recall': 0.71086971557408, 'macro_f1': 0.7108542275042378, 'micro_precision': 0.7109036345448483, 'micro_recall': 0.7109036345448483, 'micro_f1': 0.7109036345448482, 'A_precision': 0.7518518518518519, 'A_recall': 0.6835016835016835, 'A_f1': 0.7160493827160493, 'A_support': 594, 'B_precision': 0.680921052631579, 'B_recall': 0.7250437828371279, 'B_f1': 0.7022900763358779, 'B_support': 571, 'C_precision': 0.6928895612708018, 'C_recall': 0.726984126984127, 'C_f1': 0.7095274980635166, 'C_support': 630, 'D_precision': 0.7356115107913669, 'D_recall': 0.6771523178807947, 'D_f1': 0.7051724137931035, 'D_support': 604, 'E_precision': 0.7018927444794952, 'E_recall': 0.7416666666666667, 'E_f1': 0.7212317666126418, 'E_support': 600}, array([[406,  42,  50,  37,  59],
       [ 33, 414,  42,  36,  46],
       [ 33,  52, 458, 

In [11]:
results = zero_shot_qa(model, tokenizer, dev)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_baseline_results_dev.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 1500, 'answered_questions': 1500, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.7426666666666667, 'macro_precision': 0.7423361678090684, 'macro_recall': 0.7426878105939476, 'macro_f1': 0.741239059523713, 'micro_precision': 0.7426666666666667, 'micro_recall': 0.7426666666666667, 'micro_f1': 0.7426666666666667, 'A_precision': 0.7635658914728682, 'A_recall': 0.6746575342465754, 'A_f1': 0.7163636363636364, 'A_support': 292, 'B_precision': 0.7252396166134185, 'B_recall': 0.7800687285223368, 'B_f1': 0.7516556291390728, 'B_support': 291, 'C_precision': 0.7320261437908496, 'C_recall': 0.7593220338983051, 'C_f1': 0.7454242928452578, 'C_support': 295, 'D_precision': 0.8006134969325154, 'D_recall': 0.7457142857142857, 'D_f1': 0.772189349112426, 'D_support': 350, 'E_precision': 0.6902356902356902, 'E_recall': 0.7536764705882353, 'E_f1': 0.7205623901581723, 'E_support': 272}, array([[197,  21,  23,  24,  27],
       [ 11, 227,  19,  13,  21],
       [ 21,  18, 22

In [12]:
results = zero_shot_qa(model, tokenizer, test)

y_results = pd.DataFrame(results)
y_results.to_csv('csqa_zeroshot_baseline_results_test.csv',index=False)


# Run comprehensive evaluation
metrics = calculate_qa_metrics(y_results)
print(metrics)

for res in results[:20]:
    print(f"Q: {res['full_qa']}")
    print(f"Predicted answer: {res['predicted_label']}")
    print(f"Correct answer: {res['correct_label']}")
    print("-----")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

({'total_questions': 500, 'answered_questions': 500, 'unanswered_questions': 0, 'answer_rate': 1.0, 'accuracy': 0.722, 'macro_precision': 0.7260225196061267, 'macro_recall': 0.7247709944641281, 'macro_f1': 0.721756628055485, 'micro_precision': 0.722, 'micro_recall': 0.722, 'micro_f1': 0.722, 'A_precision': 0.7894736842105263, 'A_recall': 0.7009345794392523, 'A_f1': 0.7425742574257426, 'A_support': 107, 'B_precision': 0.7340425531914894, 'B_recall': 0.711340206185567, 'B_f1': 0.7225130890052357, 'B_support': 97, 'C_precision': 0.6442307692307693, 'C_recall': 0.7127659574468085, 'C_f1': 0.6767676767676768, 'C_support': 94, 'D_precision': 0.7956989247311828, 'D_recall': 0.6727272727272727, 'D_f1': 0.7290640394088671, 'D_support': 110, 'E_precision': 0.6666666666666666, 'E_recall': 0.8260869565217391, 'E_f1': 0.7378640776699029, 'E_support': 92}, array([[75,  9, 10,  3, 10],
       [ 6, 69,  9,  4,  9],
       [ 5,  6, 67,  8,  8],
       [ 8,  6, 11, 74, 11],
       [ 1,  4,  7,  4, 76]])